In [1]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import gseapy as gp

from lifelines import KaplanMeierFitter as KM
from lifelines import CoxPHFitter as cox
from matplotlib import pyplot as plt
from pandas import DataFrame as df
from scipy import stats
from IPython.display import Image

import matplotlib
import matplotlib.font_manager as fm
from matplotlib.ft2font import FT2Font
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

def get_font(*args, **kwargs):
    return FT2Font(*args, **kwargs)

fm.get_font = get_font

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
import gc
gc.collect()

422

# Global Path

In [3]:
poseidon_home = '/home/dyang-server/hdd/Yue/PROJECTS/POSEIDON/'
data_home = poseidon_home + 'data/'
gsea_home = poseidon_home + 'GSEA/'
cmap_home = data_home + 'CMAP2020_Cancer/'
shift_home = poseidon_home + 'shift_ability/'
panel_home = poseidon_home + 'panels/Figure3/'

In [4]:
gene_sets = {}
with open(gsea_home + '/gene_sets/NREC_REC_91061.gmt', 'r') as f:
    for lines in f:
        lines = lines.rstrip().split('\t')
        gene_sets[lines[0]] = lines[2:]

In [5]:
ec_color = df(index=gene_sets['NREC_profile'] + gene_sets['REC_profile'], columns=['colors'])
ec_color.loc[gene_sets['NREC_profile'], 'colors'] = 'crimson'
ec_color.loc[gene_sets['REC_profile'], 'colors'] = 'royalblue'

In [6]:
# L1000 gene info
bing_landmark = pd.read_csv(data_home + 'CMAP2020_Cancer/landmark_and_bings_L1000.csv',
                            header=0, index_col=0, sep=',', dtype={'Official NCBI gene symbol': 'str'}, converters={'Official NCBI gene symbol': None})
bing_landmark

,Official NCBI gene symbol
Official NCBI gene id,
5720,PSME1
7416,VDAC1
55847,CISD1
10174,SORBS3
25803,SPDEF
...,...
5137,PDE1C
51233,DRICH1
4340,MOG


In [7]:
sig_info = pd.read_csv(data_home + 'CMAP2020_Cancer/siginfo_beta.txt',
                            header=0, index_col='sig_id', sep='\t')

/home/dyang-server/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,3,4,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
sig_info = sig_info[sig_info['tas'] >= 0.4]

In [31]:
sig_info.columns

Index(['bead_batch', 'nearest_dose', 'pert_dose', 'pert_dose_unit',
       'pert_idose', 'pert_itime', 'pert_time', 'pert_time_unit',
       'cell_mfc_name', 'pert_mfc_id', 'nsample', 'cc_q75', 'ss_ngene', 'tas',
       'pct_self_rank_q25', 'wt', 'median_recall_rank_spearman',
       'median_recall_rank_wtcs_50', 'median_recall_score_spearman',
       'median_recall_score_wtcs_50', 'batch_effect_tstat',
       'batch_effect_tstat_pct', 'is_hiq', 'qc_pass', 'pert_id', 'pert_type',
       'cell_iname', 'det_wells', 'det_plates', 'distil_ids', 'build_name',
       'project_code', 'cmap_name', 'is_exemplar_sig', 'is_ncs_sig',
       'is_null_sig'],
      dtype='object')

In [16]:
compound_info = pd.read_csv(data_home + '/CMAP2020_Cancer/compoundinfo_beta.txt',
                            header=0, index_col=1, sep='\t')
compound_info = compound_info.groupby(level=0).first()
compound_info

,pert_id,target,moa,canonical_smiles,inchi_key,compound_aliases
cmap_name,,,,,,
1-EBIO,BRD-K70586315,NaN,NaN,CCn1c2ccccc2[nH]c1=O,CXUCKELNYMZTRT-UHFFFAOYSA-N,NaN
1-HYDROXYANTHRAQUINONE,BRD-K62620932,NaN,NaN,NaN,NaN,NaN
1-HYDROXYPHENAZINE,BRD-K66151076,NaN,NaN,NaN,NaN,NaN
1-NAPHTHYLAMINE,BRD-K05741221,NaN,NaN,NaN,NaN,NaN
1-NITRONAPHTHALENE,BRD-K72839221,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
zolpidem,BRD-K44876623,GABRA1,Benzodiazepine receptor agonist,CN(C)C(=O)Cc1c(nc2ccc(C)cn12)-c1ccc(C)cc1,ZAFYATHCZYHLPB-UHFFFAOYSA-N,NaN
zonisamide,BRD-A28095882,SCN11A,Sodium channel blocker,CN1C2CCC1CC(C2)OC(c3ccccc3)c4ccccc4N,KZFDKINRISJFCO-UHFFFAOYSA-N,NaN
zopiclone,BRD-A34309505,GABRA1,GABA receptor agonist,CN1CCN(CC1)C(=O)OC2N(C(=O)c3nccnc23)c4ccc(Cl)cn4,GBBSUAFBMRNDJC-UHFFFAOYSA-N,NaN


# Table of EC genes

In [32]:
ec_table = df(index=ec_color.index, columns=['EC_profile', 'total_shRNA_expr'])

# add the ec profile first
for g in ec_table.index:
    if g in gene_sets['NREC_profile']:
        ec_table.at[g, 'EC_profile'] = 'NR-EC'
    else:
        ec_table.at[g, 'EC_profile'] = 'R-EC'

In [33]:
sig_info_shRNA = sig_info[sig_info['pert_type'] == 'trt_sh']

In [34]:
sig_info_shRNA

,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_itime,pert_time,pert_time_unit,cell_mfc_name,pert_mfc_id,...,cell_iname,det_wells,det_plates,distil_ids,build_name,project_code,cmap_name,is_exemplar_sig,is_ncs_sig,is_null_sig
sig_id,,,,,,,,,,,,,,,,,,,,,
TAK003_PC3_96H:TRCN0000245327:-666,b7,NaN,NaN,NaN,NaN,96 h,96.0,h,PC3,TRCN0000245327,...,PC3,M11|M12,TAK003_PC3_96H_X1_B7_DUO52HI53LO|TAK003_PC3_96...,TAK003_PC3_96H_X1_B7_DUO52HI53LO:M11|TAK003_PC...,NaN,TAK,POU2F2,1,1.0,0.0
TAK001_HEKTE_96H:TRCN0000298904:-666,b7,NaN,NaN,NaN,NaN,96 h,96.0,h,HEKTE,TRCN0000298904,...,HEKTE,J22,TAK001_HEKTE_96H_X1_B7_DUO52HI53LO|TAK001_HEKT...,TAK001_HEKTE_96H_X1_B7_DUO52HI53LO:J22|TAK001_...,NaN,TAK,EIF4EBP1,1,1.0,0.0
TAK003_HEKTE_96H:TRCN0000059068:-666,b7,NaN,NaN,NaN,NaN,96 h,96.0,h,HEKTE,TRCN0000059068,...,HEKTE,E07|E08,TAK003_HEKTE_96H_X1_B7_DUO52HI53LO|TAK003_HEKT...,TAK003_HEKTE_96H_X1_B7_DUO52HI53LO:E07|TAK003_...,NaN,TAK,ERBB2IP,1,1.0,0.0
TAK001_SW480_96H:J10,b6,NaN,NaN,NaN,NaN,96 h,96.0,h,SW480,TRCN0000002173,...,SW480,J10,TAK001_SW480_96H_X1_B6_DUO52HI53LO|TAK001_SW48...,TAK001_SW480_96H_X1_B6_DUO52HI53LO:J10|TAK001_...,NaN,TAK,STK3,1,1.0,0.0
EKW001_PC3_120H:TRCN0000021242:2,f1b3,NaN,2.0,uL,2 uL,120 h,120.0,h,PC3,TRCN0000021242,...,PC3,L23,EKW001_PC3_120H_X1_F1B3_DUO52HI53LO|EKW001_PC3...,EKW001_PC3_120H_X1_F1B3_DUO52HI53LO:L23|EKW001...,NaN,EKW,MECP2,0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERGK010_VCAP_120H:TRCN0000002000:-666,b2,NaN,NaN,NaN,NaN,120 h,120.0,h,VCAP,TRCN0000002000,...,VCAP,P11,ERGK010_VCAP_120H_X1_B2_DUO52HI53LO|ERGK010_VC...,ERGK010_VCAP_120H_X1_B2_DUO52HI53LO:P11|ERGK01...,NaN,ERGK,PDGFRB,0,1.0,0.0
ERGK002_VCAP_120H:E03,b2,NaN,NaN,NaN,NaN,120 h,120.0,h,VCAP,TRCN0000000757,...,VCAP,E03,ERGK002_VCAP_120H_X1_B2_DUO52HI53LO|ERGK002_VC...,ERGK002_VCAP_120H_X1_B2_DUO52HI53LO:E03|ERGK00...,NaN,ERGK,CLK1,0,1.0,0.0
ERGK013_VCAP_120H:TRCN0000196700:-666,b3,NaN,NaN,NaN,NaN,120 h,120.0,h,VCAP,TRCN0000196700,...,VCAP,J18,ERGK013_VCAP_120H_X1_B3_DUO52HI53LO|ERGK013_VC...,ERGK013_VCAP_120H_X1_B3_DUO52HI53LO:J18|ERGK01...,NaN,ERGK,GK,1,1.0,0.0


In [35]:
# add the shRNA experiments number
for g in ec_table.index:
    if g in sig_info_shRNA['cmap_name'].unique():
        ec_table.at[g, 'total_shRNA_expr'] = sig_info_shRNA[sig_info_shRNA['cmap_name'] == g].shape[0]
    else:
        ec_table.at[g, 'total_shRNA_expr'] = 'Not available'

In [36]:
ec_table

,EC_profile,total_shRNA_expr
TYRO3,NR-EC,6
TMCC2,NR-EC,Not available
NDRG3,NR-EC,Not available
ATP9A,NR-EC,Not available
FGD1,NR-EC,Not available
...,...,...
TNFRSF1B,R-EC,Not available
CFLAR,R-EC,31
NCF4,R-EC,Not available
NSMCE4A,R-EC,Not available


In [37]:
sig_info_cp = sig_info[sig_info['pert_type'] == 'trt_cp']
sig_info_cp['target'] = 'Not available'

<ipython-input-37-7de2bdf146e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_info_cp['target'] = 'Not available'


In [38]:
for s in sig_info_cp.index:
    tmp_d = sig_info_cp.loc[s, 'cmap_name']
    if tmp_d in compound_info.index:
        sig_info_cp.at[s, 'target'] = compound_info.loc[tmp_d, 'target']

In [39]:
# add the pharmacological inhibition experiments number
for g in ec_table.index:
    if g in sig_info_cp['target'].unique():
        ec_table.at[g, 'total_pharmacological_expr'] = sig_info_cp[sig_info_cp['target'] == g].shape[0]
    else:
        ec_table.at[g, 'total_pharmacological_expr'] = 'Not available'

In [40]:
ec_table

,EC_profile,total_shRNA_expr,total_pharmacological_expr
TYRO3,NR-EC,6,Not available
TMCC2,NR-EC,Not available,Not available
NDRG3,NR-EC,Not available,Not available
ATP9A,NR-EC,Not available,Not available
FGD1,NR-EC,Not available,Not available
...,...,...,...
TNFRSF1B,R-EC,Not available,Not available
CFLAR,R-EC,31,Not available
NCF4,R-EC,Not available,Not available
NSMCE4A,R-EC,Not available,Not available


In [44]:
ec_table['shRNA_cell_lines'] = 'Not available'
ec_table['compound_cell_lines'] = 'Not available'

In [47]:
# add the cell line counts
for g in ec_table.index:
    if ec_table.loc[g, 'total_shRNA_expr'] != 'Not available':
        tmp_g_sig = sig_info_shRNA[sig_info_shRNA['cmap_name'] == g]
        tmp_g_sig_count = df(tmp_g_sig['cell_mfc_name'].value_counts())
        ec_table.at[g, 'shRNA_cell_lines'] = list(tmp_g_sig_count.index)
    if ec_table.loc[g, 'total_pharmacological_expr'] != 'Not available':
        tmp_g_sig = sig_info_cp[sig_info_cp['target'] == g]
        tmp_g_sig_count = df(tmp_g_sig['cell_mfc_name'].value_counts())
        ec_table.at[g, 'compound_cell_lines'] = list(tmp_g_sig_count.index)

In [48]:
ec_table

,EC_profile,total_shRNA_expr,total_pharmacological_expr,shRNA_cell_lines,compound_cell_lines
TYRO3,NR-EC,6,Not available,"[MCF7, VCAP, PC3, A375]",Not available
TMCC2,NR-EC,Not available,Not available,Not available,Not available
NDRG3,NR-EC,Not available,Not available,Not available,Not available
ATP9A,NR-EC,Not available,Not available,Not available,Not available
FGD1,NR-EC,Not available,Not available,Not available,Not available
...,...,...,...,...,...
TNFRSF1B,R-EC,Not available,Not available,Not available,Not available
CFLAR,R-EC,31,Not available,"[A375, A549, VCAP, HT29, HCC515, HEPG2, NPC, P...",Not available
NCF4,R-EC,Not available,Not available,Not available,Not available
NSMCE4A,R-EC,Not available,Not available,Not available,Not available


In [54]:
ec_table.to_csv(poseidon_home + 'EC_signatures/all_NREC_REC_gene_info.csv', sep=',')

In [50]:
# only show both shRNA and compound available
bitarget_table = ec_table[ec_table['total_pharmacological_expr'] != 'Not available']
bitarget_table = bitarget_table[bitarget_table['total_shRNA_expr'] != 'Not available']
bitarget_table.shape

(42, 5)

In [51]:
bitarget_table

,EC_profile,total_shRNA_expr,total_pharmacological_expr,shRNA_cell_lines,compound_cell_lines
ATP1A1,NR-EC,2,513,"[HA1E, HEPG2]","[MCF7, A549, PC3, HA1E, A375, U2OS, HT29, HCC5..."
APP,NR-EC,1,3,[A549],"[PC3, HA1E, MCF7]"
CSNK2A1,NR-EC,1,55,[A375],"[A375, HA1E, PC3, MCF7, HELA, A549, HEPG2, MCF..."
CDK2,NR-EC,6,1528,"[MCF7, HT29, U2OS, A375, SW480]","[MCF7, A549, PC3, A375, HT29, MDAMB231, MCF10A..."
PI4KB,NR-EC,2,6,[VCAP],"[SKBR3, HA1E, BT20, PC3, A375, MCF7]"
IGF1R,NR-EC,7,142,"[PC3, VCAP, HT29, HEPG2, A549]","[HT29, A375, MCF7, HCC515, MCF10A.WT, A549, HE..."
PTK2,NR-EC,21,14,"[HA1E, MCF7, PC3, VCAP, HT29, A549, A375, HEK2...","[HT29, HA1E, MCF7, PC3, A375, HELA, YAPC]"
AURKA,NR-EC,32,156,"[HEPG2, A549, A375, MCF7, HA1E, NPC, HEK293T, ...","[MCF7, A375, PC3, HA1E, A549, HEPG2, HCC515, N..."
FASN,NR-EC,2,19,"[HT29, A549]","[MCF7, HA1E, PC3, A375, HT29, RKO, A673, SKM1,..."
TYMS,NR-EC,3,63,"[A375, A549]","[MCF7, A549, PC3, A375, MCF10A, YAPC, HT29, U2..."


In [53]:
bitarget_table.to_csv(poseidon_home + 'EC_signatures/sh_cp_available_NREC_REC_gene_info.csv', sep=',')

# Correlation with TCGA

In [55]:
# corr between EC genes
skcm_corr = pd.read_csv(poseidon_home + 'results/corr_EC_TCGA/pearson_corr_SKCM.csv', header=0, index_col=0, sep=',')

In [56]:
skcm_corr

,AAAS,AACS,AAR2,ABCA1,ABCB6,ABCF1,ABCF2,ABTB2,ACAA1,ACACA,...,ZNF318,ZNF330,ZNF343,ZNF473,ZNF532,ZNF609,ZNF623,ZNF74,ZNF768,ZWILCH
0,,,,,,,,,,,,,,,,,,,,,
AAAS,1.000000,0.236276,0.424416,-0.285606,0.438673,0.259273,0.352424,-0.035436,0.279098,-0.049569,...,-0.258990,0.068324,0.048076,-0.043022,-0.063314,0.101263,-0.152364,0.280503,0.447919,-0.158950
AACS,0.236276,1.000000,0.131968,-0.057989,0.214873,-0.045781,0.297612,-0.088229,0.192619,0.155996,...,-0.315955,0.037399,-0.111298,-0.187129,-0.055083,0.081436,-0.260586,0.052418,0.180839,-0.009862
AAR2,0.424416,0.131968,1.000000,-0.248447,0.405184,0.459602,0.334302,0.194938,0.217625,0.138566,...,-0.015936,0.198518,0.229531,0.118219,0.088690,0.275127,0.058957,0.442682,0.510817,-0.397273
ABCA1,-0.285606,-0.057989,-0.248447,1.000000,-0.312898,-0.250074,-0.176501,-0.163644,-0.040916,-0.244928,...,-0.076397,-0.183227,-0.249146,-0.247034,0.151754,-0.117184,0.025425,-0.414653,-0.374302,-0.003304
ABCB6,0.438673,0.214873,0.405184,-0.312898,1.000000,0.236599,0.270337,0.143525,0.192916,0.136091,...,-0.136184,0.131749,0.125944,0.056349,-0.039160,0.050515,0.042107,0.349357,0.444040,-0.183111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF609,0.101263,0.081436,0.275127,-0.117184,0.050515,0.256953,0.130837,0.019254,-0.098940,0.199880,...,0.166489,-0.005131,0.135049,0.068845,0.101667,1.000000,0.045047,0.199144,0.150337,-0.015441
ZNF623,-0.152364,-0.260586,0.058957,0.025425,0.042107,0.157950,-0.111050,0.168558,-0.337588,0.238422,...,0.330183,0.227656,0.308882,0.368400,0.227200,0.045047,1.000000,0.233250,0.035923,0.054773
ZNF74,0.280503,0.052418,0.442682,-0.414653,0.349357,0.430215,0.290382,0.181186,-0.066233,0.309652,...,0.141776,0.274512,0.337962,0.419868,0.060441,0.199144,0.233250,1.000000,0.502109,-0.044020
